In [1]:
import numpy as np 
import pandas as pd
import random
import os

np.random.seed(45)

In [2]:
no_dup_path='../Our Labeled Data/No_Dup/no_d/'
contains_dup_path='../Our Labeled Data/Contains_Dup/contains_d/'

In [3]:
no_dup_files = os.listdir(no_dup_path)
contains_dup_files = os.listdir(contains_dup_path)

### 1. Approach to create labeled data with features - word1, word2, isDuplicate

1. Randomly pick one file from the contains_duplicate dataset
2. Randomly pick two rows
3. Populate the output data with words and isDuplicate value according to the entity number from the two pairs of rows


In this approach we completely ignore the data statistics and context information

In [4]:
df1= pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})
for _ in range(100):
    file=random.choice(contains_dup_files)
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    for _ in range(10):
        t=tempdf.sample(n=2).reset_index()
        w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
        w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
        isDup=(e1==e2)
        df1= df1.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)

### 2. Approach to create labeled data with features - word1, word2, isDuplicate

1. Randomly pick one file from the contains_duplicate dataset
2. Randomly pick two rows with a. same entitity number, b. different entitity number
3. Populate the output data with words and isDuplicate value according to the entity number from the two pairs of rows


In this approach we completely ignore the data statistics and context information

In [14]:
df2 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'isDuplicate':[]})
# df2 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

for _ in range(100):
    file=random.choice(contains_dup_files)
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    
    perEntityCatCount=tempdf['Entity_Number'].value_counts()

    for _ in range(3):
        tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
        
        try:
            t=tempdf[tempdf['entityCount']>1].sample(n=1).reset_index()
            e=t.loc[0]['Entity_Number']
            t=tempdf[tempdf['Entity_Number']==e].sample(n=2).reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'isDuplicate':isDup}, ignore_index=True)
            # df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
        except ValueError:
            pass

        try:
#             t=tempdf[tempdf['entityCount']==1].sample(n=2).reset_index()
            t=tempdf.sample(n=2).reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'isDuplicate':isDup}, ignore_index=True)
            # df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)

        except ValueError:
            pass

In [19]:
# Drop Duplicates
df2 = df2.drop_duplicates()

In [20]:
# Test

df2[df2["w1"]=="U.S. dollar"]

,w1,w2,count1,count2,isDuplicate
324,U.S. dollar,U. S. dollar,15.0,1.0,1.0


# Aproach 3

Get all Pairs of duplicates

In [21]:
df3 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

# contains_dup_path

total = 0

for file in contains_dup_files:
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    perEntityCatCount=tempdf['Entity_Number'].value_counts()
    
    tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
    
    dupEntityNo = set((tempdf[tempdf['entityCount']>1])['Entity_Number'])
    
    total+=len(dupEntityNo)
    
    for eNo in dupEntityNo:
        t = tempdf[tempdf['Entity_Number']==eNo].reset_index()
        w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
        w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
        isDup=(e1==e2)
    
        df3 = df3.append({'w1':w1.strip(), 'w2':w2.strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
    
        df3 = df3.append({'w1':w2.strip(), 'w2':w1.strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)
    
    

In [22]:
df3_nodup = df3.drop_duplicates()
df3_nodup = df3_nodup[df3_nodup['w1']!=df3_nodup['w2']].reset_index()

In [23]:
df3_nodup.to_csv('allDuplicates.csv')

Get Randomly sampled Non Duplicates

In [25]:
df4 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

total = 0

for file in no_dup_files:
    file_path=no_dup_path+file
    tempdf=pd.read_csv(file_path)
    
    for _ in range(20):
        if len(tempdf)>2:
            t=tempdf.sample(n=2)
            tempdf = tempdf.drop(t.index)
            t = t.reset_index()
            w1,c1,t1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples']
            w2,c2,t2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples']
            isDup=0.0
            
            if type(w1)==str and type(w2)==str: 
            
                df4 = df4.append({'w1':str(w1).strip(), 'w2':str(w2).strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
                df4 = df4.append({'w1':str(w2).strip(), 'w2':str(w1).strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)
                
                
for file in contains_dup_files:
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    perEntityCatCount=tempdf['Entity_Number'].value_counts()
    
    tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
    
    tempdf = tempdf[tempdf['entityCount']==1]
    
    for _ in range(20):
        if len(tempdf)>2:
            t=tempdf.sample(n=2)
            tempdf = tempdf.drop(t.index)
            t = t.reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            
            if type(w1)==str and type(w2)==str: 
                df4 = df4.append({'w1':w1.strip(), 'w2':w2.strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
                df4 = df4.append({'w1':w2.strip(), 'w2':w1.strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)

df4_nodup = df4.drop_duplicates()
df4_nodup = df4_nodup.sample(frac=1).reset_index(drop=True)

In [27]:
df4_nodup.to_csv('NoDuplicates.csv')

Combine Duplicates and Non Duplicates

In [28]:
df5 = pd.concat((df3_nodup, df4_nodup))

df5 = df5.sample(frac=1).reset_index(drop=True)

df5.to_csv('1AllDuplicates5NoDuplicates.csv')

# Aproach 4

### Hold out files from train set

In [30]:
# contains_dup_files

record_ids = []
for f in contains_dup_files:
    record_ids.append(f.split()[1])

trainRecordID = random.choices(list(set(record_ids)), k = 25)
testRecordID = list(set(record_ids) - set(trainRecordID))

In [31]:
contains_dup_file_HoldOutTrain = []
contains_dup_file_HoldOutTest = []

for f in contains_dup_files:
    if f.split()[1] in trainRecordID:
        contains_dup_file_HoldOutTrain.append(f)
    else:
        contains_dup_file_HoldOutTest.append(f)

In [32]:
dfHoldOutTrainDup = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

# contains_dup_path
total = 0
for file in contains_dup_file_HoldOutTrain:
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    perEntityCatCount=tempdf['Entity_Number'].value_counts()
    
    tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
    
    dupEntityNo = set((tempdf[tempdf['entityCount']>1])['Entity_Number'])
    
    total+=len(dupEntityNo)
    
    for eNo in dupEntityNo:
        t = tempdf[tempdf['Entity_Number']==eNo].reset_index()
        w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
        w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
        isDup=(e1==e2)
    
        dfHoldOutTrainDup = dfHoldOutTrainDup.append({'w1':w1.strip(), 'w2':w2.strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)    
        dfHoldOutTrainDup = dfHoldOutTrainDup.append({'w1':w2.strip(), 'w2':w1.strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)
    
dfHoldOutTrainDup = dfHoldOutTrainDup.drop_duplicates()

In [34]:
dfHoldOutTestDup = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

# contains_dup_path

total = 0

for file in contains_dup_file_HoldOutTest:
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    perEntityCatCount=tempdf['Entity_Number'].value_counts()
    
    tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
    
    dupEntityNo = set((tempdf[tempdf['entityCount']>1])['Entity_Number'])
    
    total+=len(dupEntityNo)
    
    for eNo in dupEntityNo:
        t = tempdf[tempdf['Entity_Number']==eNo].reset_index()
        w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
        w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
        isDup=(e1==e2)
    
        dfHoldOutTestDup = dfHoldOutTestDup.append({'w1':w1.strip(), 'w2':w2.strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
    
        dfHoldOutTestDup = dfHoldOutTestDup.append({'w1':w2.strip(), 'w2':w1.strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)
    

dfHoldOutTestDup = dfHoldOutTestDup.drop_duplicates()

Combining Duplicates and non duplicates

In [36]:
trainLen, testLen = dfHoldOutTrainDup.shape[0], dfHoldOutTestDup.shape[0]
totalLen = trainLen+testLen

HoldOutTrainData = pd.concat([dfHoldOutTrainDup, df4[:int(df4.shape[0]*(trainLen/totalLen))]])
HoldOutTestData = pd.concat([dfHoldOutTestDup, df4[int(df4.shape[0]*(trainLen/totalLen)):]])

In [38]:
HoldOutTrainData = HoldOutTrainData.drop_duplicates()
HoldOutTrainData = HoldOutTrainData.sample(frac=1).reset_index(drop=True)

HoldOutTestData = HoldOutTestData.drop_duplicates()
HoldOutTestData = HoldOutTestData.sample(frac=1).reset_index(drop=True)

HoldOutTrainData.to_csv("HoldOutTrainData.csv")
HoldOutTestData.to_csv("HoldOutTestData.csv")

In [39]:
pd.DataFrame({"trainRecordID":trainRecordID}).to_csv("trainRecordIDs.csv")

pd.DataFrame({"testRecordID":testRecordID}).to_csv("testRecordIDs.csv")